In [ ]:
# =================================================================
# 1. 環境セットアップ & ライブラリインストール
# =================================================================
# Colabの既存パッケージとの競合を避けるため、Pandasのバージョンを固定
!pip install -U -q "pandas<3.0.0" "google-genai" openpyxl

import os
import json
import pandas as pd
from google.colab import drive
from google.genai import types
from google.genai import Client

# =================================================================
# 2. Google Drive マウント & パス設定
# =================================================================
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- ここをプロジェクトのフォルダパスに書き換えてください ---
PROJECT_ROOT = '/content/drive/MyDrive/path/to/your/project' 
# --------------------------------------------------------

# .envはプロジェクト直下にあると想定
ENV_PATH = os.path.join(PROJECT_ROOT, '.env')

if os.path.exists(ENV_PATH):
    load_dotenv(ENV_PATH)
    print(f"✅ Loaded environment variables from: {ENV_PATH}")
else:
    print(f"⚠️ .env file not found at: {ENV_PATH}")

# 環境変数から取得（.env に記述された値が優先される）
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# もし .env 内で BASE_DIR を定義していない場合は PROJECT_ROOT を使う
BASE_DIR = os.getenv("BASE_DIR", PROJECT_ROOT)

INPUT_FILE = os.path.join(BASE_DIR, 'xhs_summary.txt')
OUTPUT_FILE = os.path.join(BASE_DIR, 'xhs_cohort_progress.csv')

# クライアント初期化
client = Client(api_key=GEMINI_API_KEY)

# =================================================================
# 3. 解析ロジック本体
# =================================================================
def run_cohort_analysis():
    # ファイル存在チェック
    if not os.path.exists(INPUT_FILE):
        print(f"❌ Error: {INPUT_FILE} が見つかりません。")
        print("左側のフォルダアイコンから、上記パスに解析したいテキストファイルを配置してください。")
        return

    # テキスト読み込み
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    print("🤖 Gemini AI がデータを構造化しています...")

    # 精度を最大化するための英語プロンプト（指示は厳格に）
    prompt = f"""
    Act as a Senior Data Engineer. Extract monthly cohort data from the provided report.

    [EXTRACTION LOGIC]
    1. "Month": Convert "24年X月" format to "2024-X" (e.g., 2024-01).
    2. "Total_Applied": Find the number in "⭐各月提交人数" section for that month.
    3. "Approved_Main": The number BEFORE '+' in the "下签" line.
    4. "Approved_Family": The number AFTER '+' in the "下签" line.
    5. "RFE_Count": The number of "补材料" (Request for Evidence).
    6. "Notes": Briefly summarize month-specific trends in Japanese.

    [OUTPUT RULES]
    - Return ONLY a raw JSON array.
    - No markdown formatting, no conversational text.

    [TARGET TEXT]
    {raw_text}
    """

    try:
        # 最新のSDK方式でリクエスト
        response = client.models.generate_content(
            model='gemini-2.0-flash', # 最新モデルを使用
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=0.1, # 抽出精度を高めるため低めに設定
                response_mime_type='application/json' # 出力をJSONに固定
            )
        )

        # JSONをパース
        structured_data = json.loads(response.text)

        # DataFrame化
        df = pd.DataFrame(structured_data)

        # 数値計算列の追加（Tableauでの分析を容易にする）
        df['Total_Applied'] = pd.to_numeric(df['Total_Applied'], errors='coerce')
        df['Approved_Main'] = pd.to_numeric(df['Approved_Main'], errors='coerce').fillna(0)
        df['Approved_Family'] = pd.to_numeric(df['Approved_Family'], errors='coerce').fillna(0)
        df['RFE_Count'] = pd.to_numeric(df['RFE_Count'], errors='coerce').fillna(0)

        df['Total_Approved'] = df['Approved_Main'] + df['Approved_Family']
        df['Movement_Total'] = df['Total_Approved'] + df['RFE_Count']

        # 抽出成功率（動向があった件数 / 申請総数）
        df['Activity_Rate'] = (df['Movement_Total'] / df['Total_Applied']).round(4)

        # タイムスタンプ追加
        df['Last_Updated'] = pd.Timestamp.now().strftime('%Y-%m-%d')

        # CSV保存（Excel/Tableau対策で utf-8-sig）
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

        print(f"✅ 解析完了！CSVファイルを保存しました: {OUTPUT_FILE}")
        print("-" * 30)
        print(df[['Month', 'Total_Applied', 'Total_Approved', 'RFE_Count', 'Activity_Rate']].head())

    except Exception as e:
        print(f"❌ 解析中にエラーが発生しました: {e}")

# =================================================================
# 4. 実行
# =================================================================
run_cohort_analysis()